## Approximate Nearest Neighbors
Most Collaborative Filtering algorithms (both Memory and Model-based) suffer from the
complexity of nearest neighbor lookup at industry-scale data. Use Spark’s Locality
Sensitive Hashing methods to design collaborative filtering algorithms that scale.
Assuming that you are using a single machine (i.e. your laptop, not a cluster), how does
the LSH approach compare to naive brute-force approaches in speed and accuracy?

Brute Force Algorithm Comes From <b>Scikit Leanrn:</b> https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html

Locality Sensitive Hashing Algorithm Comes From <b>Spark ML:</b> http://spark.apache.org/docs/2.2.0/api/python/pyspark.ml.html#pyspark.ml.feature.BucketedRandomProjectionLSH